In [3]:
import typing
import httpx
from rich import print
from pydantic import BaseModel


class GithubIssue(BaseModel):
    title: str
    issue_url: str


def get_github_issues(api: str, username: str) -> typing.Iterable[GithubIssue]:
    search_url = f"{api}/search/issues"
    query_params = {"q": f"commenter:{username}", "sort": "created", "order": "desc"}
    response = httpx.get(search_url, params=query_params, follow_redirects=True)
    try:
        for item in response.json()["items"]:
            yield GithubIssue(title=item["title"], issue_url= item["html_url"])
    except Exception as exc:
        print(f"Something bad happened while reading data from github: {type(exc)} => {exc.args}")
        raise SystemError("Going down...")

In [4]:
import os

import django
from django.utils import timezone


# set paths
#     1) to locate project root to enable founding of django project and its applications
#     2) to locate django project settings for django setup
os.sys.path.insert(0, os.getenv("PROJECT_ROOT", "."))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "softorks.settings")

# startup of django
django.setup()

In [5]:
from functools import partial
from asgiref.sync import sync_to_async

# import Activity model
from activities.models import Activity


github_api = os.environ.get("GITHUB_API", "https://api.github.com")
github_user = os.environ.get("GITHUB_USER", "michalnik")


# set initial data from github
for issue in get_github_issues(github_api, github_user):
    # we are in async context(natural for notebook) so we need to call sync_to_async for Django ORM
    #    - because of sync_to_async signature it is required to create new sync function to call it later without any arguments
    #    - functools.partial can do the work ...
    create_activity = partial(Activity.objects.create, created_at=timezone.now(), title=issue.title, issue_url=issue.issue_url, comment_url="")
    await sync_to_async(create_activity, thread_sensitive=True)()

GithubIssue(title='Ticket 26434', issue_url='https://github.com/django/django/pull/19524')

GithubIssue(title='Ticket 26434 2', issue_url='https://github.com/django/django/pull/19519')

GithubIssue(title='Ticket 26434', issue_url='https://github.com/django/django/pull/19504')

GithubIssue(title='Migrate to python3.13', issue_url='https://github.com/michalnik/softorks.eu/issues/6')

GithubIssue(title='Add section of references', issue_url='https://github.com/michalnik/softorks.eu/issues/5')

...

GithubIssue(
    title='optional argument --bootstrap with value no install djangocms-bootstrap4',
    issue_url='https://github.com/nephila/djangocms-installer/issues/392'
)

GithubIssue(title='[BUG]', issue_url='https://github.com/django-cms/django-cms/issues/7258')

GithubIssue(title='GLIBC_2.27 not supported', issue_url='https://github.com/atom/atom/issues/18170')

GithubIssue(
    title='Curl https issue ssl3_get_record when behind proxy',
    issue_url='https://github.com/curl/curl/issues/2324'
)

GithubIssue(
    title='"login with google" silently fails, no login occurs',
    issue_url='https://github.com/pypi/legacy/issues/492'
)